# Python OOP for Ops Tutorial

This tutorial is to demonstrate the basic principles of Object-Oriented Programming in Python in an Operations scenario. This knowledge will help you take your Operations programming skills to the next level, from basic single-purpose scripts to reusable, flexible, object-oriented code.

To get the best from this tutorial, you need to be _somewhat_ familiar with Python, to the extent that you've written some scripts to do operational tasks -- most likely using the `requests` module -- but you don't need to have written code using your own classes: that's what you will learn from this tutorial.

The core scenario of the tutorial is this: you have several remote routers and your task is to determine which of these are busiest. To achieve this, you decide to compile throughput metrics on the interfaces of each router as a proxy for "busy-ness".

## Scenario 1

In the early days, all routers are the same. A single, simple script iterates over all routers' addresses, logs in to each and pulls its traffic data. (To focus on the core lesson, the login credentials are the same for all routers. Of course, in real life, no-one would do that.)

### Set up the simulation

If you want to practice yourself outside of these notebooks, you can use the code below (or the same from Scenario 2) to set up the simulation. It creates some simulated routers in memory and provides an interface similar to that presented by the `requests` module.

In [1]:
# Load the libraries
import os, sys, json
sys.path.insert(0, "../src")
from tutor_router.scenarios import Scenario, NotFoundError

# Set up the scenario
scenario = Scenario.scenario_1()

# Make these look a bit like `requests` calls.
request = scenario.request
get = scenario.get
post = scenario.post

# Get a list of host names, for convenience.
hosts = tuple(scenario.hosts())

> NOTE: this simulation's interface is _similar to_ but not _precisely_ like `requests`. Don't use the call patterns here in real life code. Refer to the [`requests` docs](https://docs.python-requests.org/en/latest/) and tutorials for the Real Thing(tm).

These are the hosts. We can use these as hostnames.

In [2]:
hosts

('TutorRouter-1',
 'TutorRouter-2',
 'TutorRouter-3',
 'TutorRouter-4',
 'TutorRouter-5',
 'TutorRouter-6',
 'TutorRouter-7',
 'TutorRouter-8',
 'TutorRouter-9',
 'TutorRouter-10',
 'TutorRouter-11',
 'TutorRouter-12',
 'TutorRouter-13',
 'TutorRouter-14',
 'TutorRouter-15',
 'TutorRouter-16',
 'TutorRouter-17',
 'TutorRouter-18',
 'TutorRouter-19',
 'TutorRouter-20')

## Building a typical Ops script

**NOTE** keep in mind that this is a _simulation_. It is _somewhat_ like what you'd do with `requests` against real hosts, but not _exactly_ so. In order to concentrate on the core lessons, many details have been left out , particularly around how you'd acquire and use authentication tokens.

There's a healthcheck endpoint on each router. Let's poll them all first.

In [3]:
# Iterate over all routers
for hostname in scenario.hosts():

    # Perform a health check on the current router.
    health = scenario.get(hostname, "healthcheck")
    print(health)

(200, '{"host": "TutorRouter-1", "version": 1, "health": "good"}')


NotFoundError: Connection error

`ConnectionError`

Bummer, some of them are down.

Never mind, we can catch that exception and skip those routers until they are fixed.

In [4]:
# Iterate over all routers.
for hostname in hosts:
    # Perform a health check on the current router.
    try:
        health = get(hostname, "healthcheck")
    except NotFoundError:
        print(f"Host {hostname} is not available")
        continue
    print(f"Host {hostname} is up")

Host TutorRouter-1 is up
Host TutorRouter-2 is not available
Host TutorRouter-3 is up
Host TutorRouter-4 is up
Host TutorRouter-5 is up
Host TutorRouter-6 is up
Host TutorRouter-7 is up
Host TutorRouter-8 is up
Host TutorRouter-9 is up
Host TutorRouter-10 is up
Host TutorRouter-11 is up
Host TutorRouter-12 is not available
Host TutorRouter-13 is up
Host TutorRouter-14 is up
Host TutorRouter-15 is up
Host TutorRouter-16 is up
Host TutorRouter-17 is up
Host TutorRouter-18 is not available
Host TutorRouter-19 is not available
Host TutorRouter-20 is up


Cool, now we can get the interface stats.

In [5]:
# Iterate over all routers
for hostname in hosts:
    # Perform a health check on the current router.
    try:
        health = get(hostname, "healthcheck")
    except NotFoundError:
        print(f"Host {hostname} is not available")
        continue
    print(f"Host {hostname} is up")

    # Now get the interface stats.
    status_code, response = get(hostname, "interfaces")
    print(f"  interfaces: ({status_code}) {response}")

Host TutorRouter-1 is up
  interfaces: (403) {"message": "Forbidden"}
Host TutorRouter-2 is not available
Host TutorRouter-3 is up
  interfaces: (403) {"message": "Forbidden"}
Host TutorRouter-4 is up
  interfaces: (403) {"message": "Forbidden"}
Host TutorRouter-5 is up
  interfaces: (403) {"message": "Forbidden"}
Host TutorRouter-6 is up
  interfaces: (403) {"message": "Forbidden"}
Host TutorRouter-7 is up
  interfaces: (403) {"message": "Forbidden"}
Host TutorRouter-8 is up
  interfaces: (403) {"message": "Forbidden"}
Host TutorRouter-9 is up
  interfaces: (403) {"message": "Forbidden"}
Host TutorRouter-10 is up
  interfaces: (403) {"message": "Forbidden"}
Host TutorRouter-11 is up
  interfaces: (403) {"message": "Forbidden"}
Host TutorRouter-12 is not available
Host TutorRouter-13 is up
  interfaces: (403) {"message": "Forbidden"}
Host TutorRouter-14 is up
  interfaces: (403) {"message": "Forbidden"}
Host TutorRouter-15 is up
  interfaces: (403) {"message": "Forbidden"}
Host TutorRo

Ah, that function needs login. OK let's do that.

In [6]:
# Iterate over all routers.
for hostname in hosts:
    # Perform a health check on the current router.
    try:
        health = get(hostname, "healthcheck")
    except NotFoundError:
        print(f"Host {hostname} is not available")
        continue
    print(f"Host {hostname} is up")

    # We need to log in before we can do anything more than health check...
    status_code, response = post(hostname, "login", headers={"username": "admin", "password": "Password123"})
    print(f"  log in: ({status_code}) {response}")

Host TutorRouter-1 is up
  log in: (200) 65788d79-dc50-4d9b-bc9f-fee342f76d5e
Host TutorRouter-2 is not available
Host TutorRouter-3 is up
  log in: (200) 9275a916-2783-4247-b2c4-41a68e5e1489
Host TutorRouter-4 is up
  log in: (200) aff8f2b0-3141-43cc-b8a1-965bc2cece95
Host TutorRouter-5 is up
  log in: (200) 3de9f02d-f333-4354-87a4-87b7c20615e0
Host TutorRouter-6 is up
  log in: (200) 21104ed0-3b1a-47ce-bc0a-4fa07e840d94
Host TutorRouter-7 is up
  log in: (200) 4954da95-8bb1-4fc1-8c28-d91806ce7760
Host TutorRouter-8 is up
  log in: (200) 6044b541-5bd9-44d4-b34e-fec42e2dad03
Host TutorRouter-9 is up
  log in: (200) 337bf146-f9cb-414f-8ffe-f4b2d42126c8
Host TutorRouter-10 is up
  log in: (200) a7b9666e-294a-433f-a98c-ebc333d5e671
Host TutorRouter-11 is up
  log in: (200) 8fc0bf53-e63a-43db-a104-11ec9260345d
Host TutorRouter-12 is not available
Host TutorRouter-13 is up
  log in: (200) 2f814b39-3335-4f1e-9daa-1311a0301ada
Host TutorRouter-14 is up
  log in: (200) 365a645e-8944-4a0d-ae91-

The response to login (again: in this simulation) is an auth token that we have to pass in the headers with every request to a protected endpoint.

Let's do that.

Also, let's not print out the auth token any more :-)

In [7]:
# Iterate over all routers.
for hostname in hosts:
    # Perform a health check on the current router.
    try:
        health = get(hostname, "healthcheck")
    except NotFoundError:
        print(f"Host {hostname} is not available")
        continue
    print(f"Host {hostname} is up")

    # We need to log in before we can do anything more than health check...
    status_code, response = post(hostname, "login", headers={"username": "admin", "password": "Password123"})
    if status_code != 200:
        print(f"  Failed to log in")
        continue
    print(f"  Logged in")
    auth_token = response

    # Now grab the interface stats.
    # We have to pass in the auth token because this is a protected operation.
    status_code, response = get(hostname, "interfaces", headers={"Token": auth_token})
    if status_code != 200:
        print(f"  Failed to get interface stats: {response}")
        continue
    print(f"  Interfaces: ({status_code}) {response}")

Host TutorRouter-1 is up
  Logged in
  Interfaces: (200) {"eth0": {"up": 825594, "down": 514451}, "mgmt": {"up": 49504, "down": 966473}}
Host TutorRouter-2 is not available
Host TutorRouter-3 is up
  Logged in
  Interfaces: (200) {"eth0": {"up": 503084, "down": 304664}, "mgmt": {"up": 838217, "down": 207374}}
Host TutorRouter-4 is up
  Logged in
  Interfaces: (200) {"eth0": {"up": 253034, "down": 431213}, "mgmt": {"up": 986686, "down": 726367}}
Host TutorRouter-5 is up
  Logged in
  Interfaces: (200) {"eth0": {"up": 336163, "down": 144899}, "mgmt": {"up": 849370, "down": 301684}}
Host TutorRouter-6 is up
  Logged in
  Interfaces: (200) {"eth0": {"up": 423477, "down": 79312}, "mgmt": {"up": 312544, "down": 842194}}
Host TutorRouter-7 is up
  Logged in
  Interfaces: (200) {"eth0": {"up": 474467, "down": 968526}, "mgmt": {"up": 679815, "down": 890544}}
Host TutorRouter-8 is up
  Logged in
  Interfaces: (200) {"eth0": {"up": 819850, "down": 41554}, "mgmt": {"up": 159422, "down": 686595}}
H

Noice! Now we can just add the "up" and "down" for `eth0`...

In [8]:
# Iterate over all routers.
for hostname in hosts:
    # Perform a health check on the current router.
    try:
        health = get(hostname, "healthcheck")
    except NotFoundError:
        print(f"Host {hostname} is not available")
        continue
    print(f"Host {hostname} is up")

    # We need to log in before we can do anything more than health check...
    status_code, response = post(hostname, "login", headers={"username": "admin", "password": "Password123"})
    if status_code != 200:
        print(f"  Failed to log in")
        continue
    print(f"  Logged in")
    auth_token = response

    # Now grab the interface stats.
    # We have to pass in the auth token because this is a protected operation.
    status_code, response = get(hostname, "interfaces", headers={"Token": auth_token})
    if status_code != 200:
        print(f"  Failed to get interface stats")
        continue
    total_throughput = response["eth0"]["up"] + response["eth0"]["down"]
    print(f"  Throughput: {total_throughput} bits per second")

Host TutorRouter-1 is up
  Logged in


TypeError: string indices must be integers

What the heck does that mean?

Ah. Turns out the response isn't a Python object, it's string-encoded JSON. Let's fix that.

In [9]:
# Iterate over all routers.
for hostname in hosts:
    # Perform a health check on the current router.
    try:
        health = get(hostname, "healthcheck")
    except NotFoundError:
        print(f"Host {hostname} is not available")
        continue
    print(f"Host {hostname} is up")

    # We need to log in before we can do anything more than health check...
    status_code, response = post(hostname, "login", headers={"username": "admin", "password": "Password123"})
    if status_code != 200:
        print(f"  Failed to log in")
        continue
    auth_token = response

    # Now grab the interface stats.
    # We have to pass in the auth token because this is a protected operation.
    status_code, response = get(hostname, "interfaces", headers={"Token": auth_token})
    
    # Decode the JSON
    data = json.loads(response)
    
    # Calculate the throughput.
    total_throughput = data["eth0"]["up"] + data["eth0"]["down"]
    print(f"  Throughput: {total_throughput} bits per second")

Host TutorRouter-1 is up
  Throughput: 1340045 bits per second
Host TutorRouter-2 is not available
Host TutorRouter-3 is up
  Throughput: 807748 bits per second
Host TutorRouter-4 is up
  Throughput: 684247 bits per second
Host TutorRouter-5 is up
  Throughput: 481062 bits per second
Host TutorRouter-6 is up
  Throughput: 502789 bits per second
Host TutorRouter-7 is up
  Throughput: 1442993 bits per second
Host TutorRouter-8 is up
  Throughput: 861404 bits per second
Host TutorRouter-9 is up
  Throughput: 1151729 bits per second
Host TutorRouter-10 is up
  Throughput: 1487490 bits per second
Host TutorRouter-11 is up
  Throughput: 1047225 bits per second
Host TutorRouter-12 is not available
Host TutorRouter-13 is up
  Throughput: 1166669 bits per second
Host TutorRouter-14 is up
  Throughput: 336081 bits per second
Host TutorRouter-15 is up
  Throughput: 1078427 bits per second
Host TutorRouter-16 is up
  Throughput: 670324 bits per second
Host TutorRouter-17 is up
  Throughput: 105317

Job done!

But wait...

Onto [Scenario 2](scenario_2.ipynb)...